In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, mean, max, min, variance, first, last, year, month, collect_list, collect_set, desc, avg

# Inicializar a sessão Spark
spark = SparkSession.builder.appName("Agregacao").getOrCreate()

# Carregar o arquivo Parquet no DataFrame 'df_video'
df_video = spark.read.parquet("/content/videos-preparados.snappy.parquet")

# Exibir os primeiros registros do DataFrame
print("Visualizando os primeiros registros do DataFrame:")
df_video.show(10)  # Exibe os primeiros 10 registros

# Exibir o esquema do DataFrame
print("Estrutura do DataFrame:")
df_video.printSchema()

# Contar o número total de registros
total_registros = df_video.count()
print(f"Total de registros no DataFrame: {total_registros}")

# Exibir as estatísticas básicas de todas as colunas numéricas
print("Estatísticas básicas do DataFrame:")
df_video.describe().show()

# Exibir as colunas disponíveis no DataFrame
print("Colunas disponíveis no DataFrame:")
print(df_video.columns)

# Identificar valores nulos por coluna
print("Contagem de valores nulos por coluna:")
df_video.select([count(col(c).isNull().cast("int")).alias(c) for c in df_video.columns]).show()


In [ ]:
#  Calcular a quantidade de registros para cada valor único da coluna "Keyword"
keyword_counts = df_video.groupBy("Keyword").agg(count("Keyword").alias("Count"))
print("Quantidade de registros por Keyword:")
keyword_counts.show()

In [ ]:
#  Calcular a média da coluna "Interaction" para cada valor único da coluna "Keyword"
keyword_mean_interaction = df_video.groupBy("Keyword").agg(mean("Interaction").alias("Mean Interaction"))
print("Média de Interaction por Keyword:")
keyword_mean_interaction.show()

In [ ]:
#  Calcular o valor máximo da coluna "Interaction" para cada valor único da coluna "Keyword"
# e ordenar pela nova coluna "Rank Interactions" em ordem decrescente
keyword_max_interaction = df_video.groupBy("Keyword").agg(max("Interaction").alias("Rank Interactions")).orderBy(desc("Rank Interactions"))
print("Máximo de Interaction por Keyword ordenado por Rank Interactions:")
keyword_max_interaction.show()

In [ ]:
#  Calcular a média e a variância da coluna "Views" para cada valor único da coluna "Keyword"
keyword_views_stats = df_video.groupBy("Keyword").agg(
    mean("Views").alias("Mean Views"),
    variance("Views").alias("Variance Views")
)
print("Média e Variância de Views por Keyword:")
keyword_views_stats.show()

In [ ]:
#  Calcular a média, o valor mínimo e o valor máximo de "Views" para cada valor único da coluna "Keyword", sem casas decimais
from pyspark.sql.functions import round
keyword_views_summary = df_video.groupBy("Keyword").agg(
    round(mean("Views")).alias("Mean Views"),
    round(min("Views")).alias("Min Views"),
    round(max("Views")).alias("Max Views")
)
print("Resumo de Views por Keyword (Média, Mínimo, Máximo):")
keyword_views_summary.show()

In [ ]:
#  Mostrar o primeiro e o último "Published At" para cada valor único da coluna "Keyword"
keyword_published_at = df_video.groupBy("Keyword").agg(
    first("Published At").alias("First Published At"),
    last("Published At").alias("Last Published At")
)
print("Primeiro e Último Published At por Keyword:")
keyword_published_at.show()

In [ ]:
#  Contar todos os "title" de forma normal e todos os únicos e verificar se há diferença
normal_title_count = df_video.select("title").count()
unique_title_count = df_video.select("title").distinct().count()
title_difference = normal_title_count != unique_title_count
print(f"Quantidade total de titles: {normal_title_count}")
print(f"Quantidade de titles únicos: {unique_title_count}")
print(f"Há diferença entre total e únicos: {title_difference}")

In [ ]:
#  Mostrar a quantidade de registros ordenados por ano em ordem ascendente
records_per_year = df_video.withColumn("Year", year("Published At"))\
    .groupBy("Year").count().orderBy("Year")
print("Quantidade de registros por Ano:")
records_per_year.show()

In [ ]:
#  Mostrar a quantidade de registros ordenados por ano e mês em ordem ascendente
records_per_year_month = df_video.withColumn("Year", year("Published At"))\
    .withColumn("Month", month("Published At"))\
    .groupBy("Year", "Month").count().orderBy("Year", "Month")
print("Quantidade de registros por Ano e Mês:")
records_per_year_month.show()

In [ ]:
#  Calcular a média acumulativa de "Likes" para cada "Keyword" ao longo dos anos
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as Fsum

windowSpec = Window.partitionBy("Keyword").orderBy("Year").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df_with_year = df_video.withColumn("Year", year("Published At"))
cumulative_likes = df_with_year.groupBy("Keyword", "Year").agg(mean("Likes").alias("Yearly Mean Likes"))\
    .withColumn("Cumulative Mean Likes", Fsum("Yearly Mean Likes").over(windowSpec))
print("Média acumulativa de Likes por Keyword ao longo dos anos:")
cumulative_likes.show()